In [1]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from torch import nn


In [2]:
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.005
batch_size = 64
num_epochs = 2

In [3]:
class RNN_LSTM(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        x = x.squeeze(1)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        x = x.squeeze(1)
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        x = x.squeeze(1)
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def prepare_data(self):
        self.train_dataset = MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
        self.test_dataset = MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)

    def train_dataloader(self):
        # REQUIRED
        return DataLoader(dataset=self.train_dataset, batch_size=batch_size, shuffle=True)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(dataset=self.train_dataset, batch_size=batch_size, shuffle=True)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(dataset=self.test_dataset, batch_size=batch_size, shuffle=True)


In [5]:
model = RNN_LSTM(input_size, hidden_size, num_layers, num_classes)   
# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus = 1,max_epochs=5)    
trainer.fit(model)

:33, 26.92it/s, loss=0.046, v_num=5]
Epoch 4:  50%|█████     | 938/1876 [00:34<00:34, 26.86it/s, loss=0.034, v_num=5]
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 1876/1876 [00:47<00:00, 39.48it/s, loss=0.034, v_num=5]


1

In [6]:
trainer.test()

Testing:  99%|█████████▉| 929/938 [00:08<00:00, 107.18it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(0.0168, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 938/938 [00:08<00:00, 105.49it/s]


[{'test_loss': 0.016790278255939484}]